# CDC Project 2025

In [ ]:
!pip install pandas
!pip install numpy

In [2]:
import pandas as pd

df = pd.read_csv("PS_2025.09.20_08.13.16.csv")

df.shape

(38898, 92)

In [3]:
# Data Filtering
df_cut = df.dropna(subset=["pl_orbper", "pl_orbsmax", "pl_eqt", "pl_insol",'pl_orbeccen','st_rad'])
print(df_cut.isnull().sum().to_string())


pl_name                0
hostname               0
default_flag           0
sy_snum                0
sy_pnum                0
discoverymethod        0
disc_year              0
disc_facility          0
soltype                0
pl_controv_flag        0
pl_refname             0
pl_orbper              0
pl_orbpererr1          6
pl_orbpererr2          6
pl_orbperlim           0
pl_orbsmax             0
pl_orbsmaxerr1     10812
pl_orbsmaxerr2     10812
pl_orbsmaxlim          0
pl_rade               68
pl_radeerr1           72
pl_radeerr2           72
pl_radelim            68
pl_radj               68
pl_radjerr1           72
pl_radjerr2           72
pl_radjlim            68
pl_bmasse          10863
pl_bmasseerr1      10917
pl_bmasseerr2      10917
pl_bmasselim       10863
pl_bmassj          10863
pl_bmassjerr1      10917
pl_bmassjerr2      10917
pl_bmassjlim       10863
pl_bmassprov       10863
pl_orbeccen            0
pl_orbeccenerr1    11177
pl_orbeccenerr2    11177
pl_orbeccenlim         0


In [4]:
isn = df_cut.isnull().sum().to_string()
print(isn)
df_copy = df_cut

for i in isn.to_string().split("\n"):
    i = i.split(" ")
    if int(i[-1]) >= 8000:
      df_copy = df_copy.drop([i[0]],axis=1)

#orbital period, orbit semi-major axis, insulation flux, equilibrium temperature

pl_name                0
hostname               0
default_flag           0
sy_snum                0
sy_pnum                0
discoverymethod        0
disc_year              0
disc_facility          0
soltype                0
pl_controv_flag        0
pl_refname             0
pl_orbper              0
pl_orbpererr1          6
pl_orbpererr2          6
pl_orbperlim           0
pl_orbsmax             0
pl_orbsmaxerr1     10812
pl_orbsmaxerr2     10812
pl_orbsmaxlim          0
pl_rade               68
pl_radeerr1           72
pl_radeerr2           72
pl_radelim            68
pl_radj               68
pl_radjerr1           72
pl_radjerr2           72
pl_radjlim            68
pl_bmasse          10863
pl_bmasseerr1      10917
pl_bmasseerr2      10917
pl_bmasselim       10863
pl_bmassj          10863
pl_bmassjerr1      10917
pl_bmassjerr2      10917
pl_bmassjlim       10863
pl_bmassprov       10863
pl_orbeccen            0
pl_orbeccenerr1    11177
pl_orbeccenerr2    11177
pl_orbeccenlim         0


AttributeError: 'str' object has no attribute 'to_string'

In [26]:
pip install scikit-learn

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
import itertools

df_copy=df_cut
target = 'pl_rade'
features = ['pl_eqt', 'pl_insol', 'st_mass', 'st_rad', 'pl_orbeccen','pl_orbsmax','pl_orbper']

df_copy = df_copy[df_copy['pl_rade']<100]
print(df_copy.shape)
feature_combos = []
for r in range(1, len(features) + 1):
    combos = itertools.combinations(features, r)
    feature_combos.extend(combos)

m = 1000000000000
f = []
for features in feature_combos:
  features = list(features)
  df_curr = df_copy[features + [target]].dropna()

  X = df_curr[features]
  y = df_curr[target]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  rf = RandomForestRegressor(n_estimators=100, random_state=43)
  rf.fit(X_train, y_train)

  y_pred = rf.predict(X_test)
  difference_percent = y_pred*100/y_test
  rmse = root_mean_squared_error(y_test, y_pred)
  if rmse < m:
    m = rmse
    f = features
  print("RMSE:", rmse,features)

print("BEST:")
print(m,f)


(11393, 92)
RMSE: 4.137064642397812 ['pl_eqt']


KeyboardInterrupt: 

In [7]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error
import numpy as np
target = 'pl_rade'
features = ['pl_eqt', 'pl_insol', 'pl_orbeccen', 'pl_orbsmax', 'pl_orbper']
df_curr = df_copy[features + [target]].dropna()
df_curr = df_curr[df_curr['pl_rade']<50]
df_curr = df_curr[df_curr['pl_orbsmax']<10]
df_curr = df_curr[df_curr['pl_insol']<8000]
df_curr= df_curr[df_curr['pl_orbper']<500]


X = df_curr[features]
y = df_curr[target]
y_log = np.log1p(y)


X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

rf = RandomForestRegressor(n_estimators=100, random_state=43)
rf.fit(X_train, y_train)



rf_pred = rf.predict(X_test)
rf_pred = np.expm1(rf_pred)
y_test = np.expm1(y_test)

rf_rmse = root_mean_squared_error(y_test, rf_pred)
mae = mean_absolute_error(y_test, rf_pred)

print("Mean Absolute Error:",mae)
print("RF RMSE:", rf_rmse, features)


Mean Absolute Error: 0.5719912558430317
RF RMSE: 1.6621532648838906 ['pl_eqt', 'pl_insol', 'pl_orbeccen', 'pl_orbsmax', 'pl_orbper']


In [34]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
import numpy as np

features = ['pl_eqt', 'pl_insol', 'pl_orbeccen', 'pl_orbsmax', 'pl_orbper']
target = 'pl_rade'
df_curr = df_copy[features + [target]].dropna()
X = df_curr[features]
y = df_curr[target]
print(df_curr.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


rf = RandomForestRegressor(random_state=42)

param_dist = {
    "n_estimators": [100, 200, 300, 500, 800, 1000],
    "max_depth": [None, 5, 10, 20, 30, 50],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["auto", "sqrt", "log2"]
}

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=30,
    cv=3,
    verbose=2,
    n_jobs=-1,
    scoring="neg_root_mean_squared_error",
    random_state=42
)



random_search.fit(X_train, y_train)

best_rf = random_search.best_estimator_

y_pred = best_rf.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)

print("Best Parameters:", random_search.best_params_)
print("Test RMSE:", rmse)

y = best_rf.predict(2000)
print()


(11393, 6)
Fitting 3 folds for each of 30 candidates, totalling 90 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
24 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/_

Best Parameters: {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None}
Test RMSE: 2.021409161356049


In [5]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
import numpy as np

features = ['pl_eqt', 'pl_insol', 'pl_orbeccen', 'pl_orbsmax', 'pl_orbper']
target = 'pl_rade'

df_curr = df_copy[features + [target]].dropna()
df_curr = df_curr[df_curr['pl_rade']<100]
X = df_curr[features]
y = df_curr[target]


y_log = np.log1p(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

rf = RandomForestRegressor(n_estimators=100, random_state=43)
rf.fit(X_train, y_train)

y_pred_log = rf.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

rmse = root_mean_squared_error(y_true, y_pred)

print("Test RMSE:", rmse)


Test RMSE: 3.702735850988512


In [14]:
from sklearn.ensemble import StackingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.metrics import root_mean_squared_error
import numpy as np
import xgboost
features = ['pl_eqt', 'pl_insol', 'pl_orbeccen', 'pl_orbsmax', 'pl_orbper']
target = 'pl_rade'
df_curr = df_copy[features + [target]].dropna()
df_curr = df_curr[df_curr['pl_rade']<40]
X = df_curr[features]
y = df_curr[target]


y_log = np.log1p(y)


X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

rf = RandomForestRegressor(n_estimators=300, max_depth=20, random_state=42)
xgb = xgboost.XGBRegressor(n_estimators=300, learning_rate=0.05, random_state=42)
svr = make_pipeline(StandardScaler(), SVR(kernel="rbf", C=100, gamma="scale"))

stack = StackingRegressor(
    estimators=[('rf', rf), ('xgb', xgb), ('svr', svr)],
    final_estimator=RidgeCV(),
    n_jobs=-1
)

stack.fit(X_train, y_train)

y_pred_log = stack.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)


rmse = root_mean_squared_error(y_true, y_pred)
print("Stacking Ensemble RMSE:", rmse)


KeyboardInterrupt: 